In [1]:
import tensorflow as tf
import aifc
import os
import numpy as np
from matplotlib import mlab
from os import listdir
from os.path import isfile, join


In [2]:
labels_path = "C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train.csv"
train_path = "C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train"

In [3]:
audiofiles = [os.path.join(train_path, f) for f in listdir(train_path) if isfile(join(train_path, f))]
audiofiles[0:5]

['C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train\\train1.aiff',
 'C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train\\train10.aiff',
 'C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train\\train100.aiff',
 'C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train\\train1000.aiff',
 'C:\\Users\\jorge\\DatasetsTFM\\KaggleData\\train\\train10000.aiff']

In [29]:
f = aifc.open(audiofiles[0])
print(f.getframerate())
f.close()

2000


In [5]:
def ReadAIFF(file):
    s = aifc.open(file,'r')
    nFrames = s.getnframes()
    strSig = s.readframes(nFrames)
    return np.fromstring(strSig, np.short).byteswap()

s = ReadAIFF(audiofiles[0])

params = {'NFFT':256, 'Fs':2000, 'noverlap':192}

P, freqs, bins = mlab.specgram(s, **params)

c:\users\jorge\pycharmprojects\audioextraction\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


In [6]:
P.shape

(129, 59)

In [15]:
import csv
import pandas as pd
# import tensorflow as tf

df = pd.read_csv(labels_path)
display(df.head())

def get_labels(labels_path):
    labels = dict()
    with open(labels_path, 'r') as f:
        reader = csv.reader(f, dialect='excel')
        for row in reader:
            labels[row[0]] = row[1]
    return labels

labels_dict = get_labels(labels_path)

,clip_name,label
0,train1.aiff,0
1,train2.aiff,0
2,train3.aiff,0
3,train4.aiff,0
4,train5.aiff,0


In [9]:
audiofiles[0].split("\\")[-1]

'train1.aiff'

In [10]:
p = 0.7
s[:int(s.shape[0] * p)].shape, s[int(s.shape[0] * (1-p)/2): int(s.shape[0] * (1+p)/2)].shape, s[int(s.shape[0] * (1-p)):].shape

((2800,), (2800,), (2800,))

In [21]:
vfunc = np.vectorize(lambda x: labels_dict[x.split('\\')[-1]])

In [24]:
(vfunc(audiofiles) == '0').sum(), (vfunc(audiofiles) == '1').sum()

(22973, 7027)

In [26]:
from sklearn.model_selection import train_test_split

X = np.array(audiofiles)
Y = vfunc(audiofiles)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)
X_train.shape, X_test.shape, (Y_train=='1').sum(), (Y_train=='0').sum()

((21000,), (9000,), 4931, 16069)

In [30]:
def get_spects(onlyfiles, labels):
    sps = []
    y = []
    for file_path in onlyfiles:
        s = ReadAIFF(file_path)
        y.append(labels[file_path.split("\\")[-1]])
        params = {'NFFT':256, 'Fs':2000, 'noverlap':192}
        P, freqs, bins = mlab.specgram(s, **params)
        sps.append(P)
    return np.array(sps), np.array(y)

X, Y = get_spects(X_test, labels_dict)
X.shape, Y.shape

c:\users\jorge\pycharmprojects\audioextraction\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


((9000, 129, 59), (9000,))

In [33]:
'Tamaño en memoria de los datos aprox: %.2fGB' % (a.nbytes/2**10/2**10/2**10)

'Tamaño en memoria de los datos aprox: 1.70GB'

In [147]:
def get_spects_enhanced(onlyfiles, labels, p=0.7, cut= True):
    if cut:
        top_hz = 40
    sps = []
    y = []
    for file_path in onlyfiles:
        s = ReadAIFF(file_path)
        this_label = int(labels[file_path.split("\\")[-1]])
        s1 = s[:int(s.shape[0] * p)]
        y.append(this_label)
        s2 = s[int(s.shape[0] * (1-p)/2): int(s.shape[0] * (1+p)/2)]
        y.append(this_label)
        s3 = s[int(s.shape[0] * (1-p)):]
        y.append(this_label)
        params = {'NFFT':256, 'Fs':2000, 'noverlap':192}
        P1, freqs, bins = mlab.specgram(s1, **params)
        sps.append(P1[:top_hz,:])
        P2, freqs, bins = mlab.specgram(s2, **params)
        sps.append(P2[:top_hz,:])
        P3, freqs, bins = mlab.specgram(s3, **params)
        sps.append(P3[:top_hz,:])
    return np.array(sps), np.array(y)


X, Y = get_spects_enhanced(audiofiles, labels_dict)
X.shape, Y.shape

c:\users\jorge\pycharmprojects\audioextraction\tfenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


((90000, 40, 40), (90000,))

In [191]:
(Y=='1').sum(), (Y=='0').sum()

(21081, 68919)

In [179]:
def enhance_with_noise(X, Y, noise_p=0.28):
    whale_index, = np.where(Y == '1')
    no_whale_index, = np.where(Y == '0')
    X_enhanced = []
    Y_enhanced = []
    for s_i in whale_index:
        new_X = X[s_i] + noise_p*X[np.random.choice(no_whale_index)]
        X_enhanced.append(new_X)
        Y_enhanced.append(1)
    return np.array(X_enhanced),  np.array(Y_enhanced)

X_enhanced, Y_enhanced = enhance_with_noise(X, Y)
X_enhanced.shape, Y_enhanced.shape

((21081, 40, 40), (21081,))

In [212]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)
X_train, Y_train = np.concatenate([X_train, X_enhanced]), np.concatenate([Y_train, Y_enhanced])
X_train.shape, X_test.shape, (Y_train=='1').sum(), (Y_train=='0').sum()

((84081, 40, 40), (27000, 40, 40), 35996, 48085)

In [ ]:
opt = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


In [213]:
Y_train, Y_test = Y_train.astype(int), Y_test.astype(int)
Y_train, Y_test = tf.keras.utils.to_categorical(Y_train, 2), tf.keras.utils.to_categorical(Y_test, 2) 

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(20, kernel_size=(7, 7), activation=tf.nn.relu, input_shape=X_train.shape[1:], name='Conv1'),
    tf.keras.layers.Dropout(0.2),  
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(40, kernel_size=(7, 7), activation=tf.nn.relu, name='Conv2'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu, name="Dense1"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax, name="Softmax")
    ])
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5, verbose=1)
model.evaluate(X_test, Y_test)

Epoch 1/5
84081/84081 [==============================] - 193s 2ms/sample - loss: 6.8600 - acc: 0.5719
Epoch 2/5
84081/84081 [==============================] - 195s 2ms/sample - loss: 6.8602 - acc: 0.5719
Epoch 3/5
84081/84081 [==============================] - 208s 2ms/sample - loss: 7.0057 - acc: 0.5621
Epoch 4/5
84081/84081 [==============================] - 217s 3ms/sample - loss: 6.8374 - acc: 0.5729
Epoch 5/5
27000/27000 [==============================] - 20s 753us/sample - loss: 3.6606 - acc: 0.7716


[3.660609540515476, 0.77155554]